# Notes: this Java notebook requires Ganymede 
* Ganymede (Java kernel for Jupyter): [Installation and documentation](https://github.com/allen-ball/ganymede)
* We need additional libraries for lucene

In [11]:
%%pom
dependencies:
- org.apache.lucene:lucene-core:9.7.0
- org.apache.lucene:lucene-analysis-common:9.7.0
- org.apache.lucene:lucene-queryparser:9.7.0

#### Common imports (java)

In [12]:
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.io.StringReader;
import java.nio.file.Files;
import java.nio.file.Paths;

import java.util.Arrays;
import java.util.ArrayList;
import java.util.HashMap;
import java.util.Map;
import java.util.List;

#### Common imports (lucene)

In [57]:
import org.apache.lucene.analysis.Analyzer;
import org.apache.lucene.analysis.CharArraySet;
import org.apache.lucene.analysis.FilteringTokenFilter;
import org.apache.lucene.analysis.TokenStream;
import org.apache.lucene.analysis.Tokenizer;
import org.apache.lucene.analysis.en.EnglishAnalyzer;
import org.apache.lucene.analysis.en.EnglishPossessiveFilter;
import org.apache.lucene.analysis.en.KStemFilter;
import org.apache.lucene.analysis.en.PorterStemFilter;
import org.apache.lucene.analysis.standard.StandardAnalyzer;
import org.apache.lucene.analysis.standard.StandardTokenizer;
import org.apache.lucene.analysis.tokenattributes.CharTermAttribute;
import org.apache.lucene.document.Document;
import org.apache.lucene.document.Field;
import org.apache.lucene.document.Field.Store;
import org.apache.lucene.document.TextField;
import org.apache.lucene.document.IntField;
import org.apache.lucene.document.FieldType;
import org.apache.lucene.document.FloatField;
import org.apache.lucene.document.FloatPoint;
import org.apache.lucene.index.DirectoryReader;
import org.apache.lucene.index.Fields;
import org.apache.lucene.index.IndexOptions;
import org.apache.lucene.index.IndexReader;
import org.apache.lucene.index.IndexWriter;
import org.apache.lucene.index.IndexWriterConfig;
import org.apache.lucene.index.LogDocMergePolicy;
import org.apache.lucene.index.MergePolicy;
import org.apache.lucene.index.MultiTerms;
import org.apache.lucene.index.NoMergePolicy;
import org.apache.lucene.index.PostingsEnum;
import org.apache.lucene.index.StoredFields;
import org.apache.lucene.index.Term;
import org.apache.lucene.index.Terms;
import org.apache.lucene.index.TermsEnum;
import org.apache.lucene.index.TermVectors;
import org.apache.lucene.search.BooleanQuery;
import org.apache.lucene.search.BooleanClause;
import org.apache.lucene.search.BooleanClause.Occur;
import org.apache.lucene.search.BoostQuery;
import org.apache.lucene.search.IndexSearcher;
import org.apache.lucene.search.Query;
import org.apache.lucene.search.ScoreDoc;
import org.apache.lucene.search.TermRangeQuery;
import org.apache.lucene.search.TermQuery;
import org.apache.lucene.search.TopDocs;
import org.apache.lucene.store.Directory;
import org.apache.lucene.store.FSDirectory;
import org.apache.lucene.store.Directory;
import org.apache.lucene.store.FSDirectory;
import org.apache.lucene.queryparser.classic.MultiFieldQueryParser;
import org.apache.lucene.queryparser.classic.QueryParser;
import org.apache.lucene.queryparser.classic.ParseException;
import org.apache.lucene.util.BytesRef;

## Define paths to documents and index

In [14]:
var fileImdbDataset = "datasets/imdb_top_1000.csv";
var pathIndex = "lucene/index";

## Let's read in the data collection

In [15]:
List<Map<String, String>> readCollection(String name) throws IOException {
    List<Map<String, String>> docs = new ArrayList<Map<String, String>>();
    String splitter = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)";
    BufferedReader reader = new BufferedReader(new FileReader(name));
    String line, keys[] = reader.readLine().split(splitter);

    while ((line = reader.readLine()) != null) {
        String[] values = line.split(splitter);
        Map<String, String> dataMap = new HashMap<>();

        for (int i = 0; i < keys.length; i++) {
            // dataMap.put(keys[i], values[i]);
            switch (keys[i]) {
                case "Series_Title":
                    dataMap.put("title", values[i]);
                    break;
                case "Released_Year":
                    dataMap.put("year", values[i]);
                    break;
                case "Runtime":
                    dataMap.put("runtime", values[i].replace(" min", ""));
                    break;
                case "Genre":
                    dataMap.put("genre", values[i].replace(",", ""));
                    break;
                case "IMDB_Rating":
                    dataMap.put("rating", values[i]);
                    break;
                case "Overview":
                    dataMap.put("summary", values[i].replace("\"", ""));
                    break;
                case "Star1":
                    dataMap.put("actors", values[i]);
                    break;
                case "Star2":
                case "Star3":
                case "Star4":
                    dataMap.put("actors", dataMap.get("actors") + " " + values[i]);
                    break;
            }
        }
        docs.add(dataMap);
    }
    reader.close();

    // print summary
    System.out.println("Read " + docs.size() + " documents from " + name);
    return docs;
}

var collection = readCollection(fileImdbDataset);
System.out.println("\nfirst document:");
collection.get(0).forEach((key, value) -> System.out.println(String.format("%10s: %s", key, value)));

Read 1000 documents from datasets/imdb_top_1000.csv

first document:
   summary: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.
    actors: Tim Robbins Morgan Freeman Bob Gunton William Sadler
      year: 1994
     genre: Drama
    rating: 9.3
   runtime: 142
     title: The Shawshank Redemption


## Analyzer of Lucene

In [16]:
void print_tokens(Analyzer analyzer, String text) throws IOException {
    TokenStream ts = analyzer.tokenStream("text", new StringReader(text));
    CharTermAttribute termAtt = ts.addAttribute(CharTermAttribute.class);

    for(ts.reset(); ts.incrementToken();) 
        System.out.print(termAtt.toString() + " ");
    ts.end();
    System.out.println();
}

class MyAnalyzer extends Analyzer {
    @Override
    protected TokenStreamComponents createComponents(String fieldName) {
          final Tokenizer source = new StandardTokenizer();
          TokenStream result = new EnglishPossessiveFilter(source);
          // result = new LowerCaseFilter(result);
          result = new FilteringTokenFilter(result) {
              private final CharTermAttribute termAtt = addAttribute(CharTermAttribute.class);
              @Override
              protected boolean accept() throws IOException {
                  return termAtt.length() > 3;
              }
          };
          result = new KStemFilter(result);
          return new TokenStreamComponents(source, result);
    }
}

var text = "I think text's values' color goes here; WHAT happens with it? do we see IT again; I went there to be gone with houses";
var stopWords = new CharArraySet(Arrays.asList("i", "do"), false);

System.out.println("             text: "+ text);
System.out.println();

// standard analyzer
System.out.print("         standard: ");
print_tokens(new StandardAnalyzer(), text);

// english analyzer (with porter stemmer)
System.out.print("          english: ");
print_tokens(new EnglishAnalyzer(), text);

// english analyzer (with porter stemmer) and new set of stopwords
System.out.print("english/stopwords: ");
print_tokens(new EnglishAnalyzer(stopWords), text);

// a custom analyzer, no lower case and kstemmer
System.out.print("      my analyzer: ");
print_tokens(new MyAnalyzer(), text);

// print standard stop word list
System.out.println("\nenglish stopword list:");
System.out.println(EnglishAnalyzer.getDefaultStopSet());

             text: I think text's values' color goes here; WHAT happens with it? do we see IT again; I went there to be gone with houses

         standard: i think text's values color goes here what happens with it do we see it again i went there to be gone with houses 
          english: i think text valu color goe here what happen do we see again i went gone hous 
english/stopwords: think text valu color goe here what happen with it we see it again went there to be gone with hous 
      my analyzer: think text value color go here WHAT happen with again went there gone with house 

english stopword list:
[but, be, with, such, then, for, no, will, not, are, and, their, if, this, on, into, a, or, there, in, that, they, was, is, it, an, the, as, at, these, by, to, of]


## Building an index with Lucene

### Providing default analyzer, diretory, and index writer/searcher
We pick the English Analyzer. Make sure that you always use the *same* analyzer for indexing and searching. Lucene is not checking for this and your search performance can suffer. The directory is on the file system. Using standard configurations.

In [17]:
Analyzer getAnalyzer() {
    return new EnglishAnalyzer();
}

Directory getDirectory() throws IOException {
    return FSDirectory.open(Paths.get(pathIndex));
}

IndexWriter getIndexWriter() throws IOException {
    Directory directory = getDirectory();
    IndexWriterConfig config = new IndexWriterConfig(getAnalyzer());
    return new IndexWriter(directory, config);
}

IndexWriter getIndexWriter(boolean mergePolicy) throws IOException {
    Directory directory = getDirectory();
    IndexWriterConfig config = new IndexWriterConfig(getAnalyzer());
    MergePolicy policy = mergePolicy ? new LogDocMergePolicy() : NoMergePolicy.INSTANCE;
    if (mergePolicy) policy.setNoCFSRatio(1.0);
    config.setMergePolicy(policy);
    return new IndexWriter(directory, config);
}

### We delete the index first, to load all documents into a fresh index

In [18]:
void deleteIndex() throws IOException {
    IndexWriter writer = getIndexWriter();
    writer.deleteAll();
    writer.commit();
    writer.close();
}

deleteIndex();

### Lucene accepts documents with fields
`Field.Store.YES` stores the value of the field in the index. That mean when we print results, we have these attributes available form the Lucene index. On the other hand, `Field.Store.NO` does not dtore the values in the index. In our example below, we have to retrieve actors, genre, and summary from the original data file as they are not stored in the Lucene index. 

The field type decides whether its value is tokenized and available for full-text search:
* `TextField`: Reader or String indexed for full-text search
* `StringField`: String indexed verbatim as a single token
* `IntField`: int indexed for exact/range queries. 
* `IntPoint`: faster int indexed for exact/range queries. If you need to store the value, also use a `StoredField` 
* `FloatField`: float indexed for exact/range queries.
* `FloatPoint`: faster float indexed for exact/range queries. If you need to store the value, also use a `StoredField` 
* `StoredField`: Stored-only value for retrieving in summary results

In summary, a field can be 'stored' / 'not stored' in the index, and the contents of a field can be used for full-text search, exact/range queries, or not at all.

In [19]:
Document createDocument(Map<String, String> data) {
    Document doc = new Document();

    // we store everything we need for result presentation
    doc.add(new TextField("title", data.get("title"), Store.YES));
    doc.add(new TextField("year", data.get("year"), Store.YES));
//    doc.add(new IntField("year", Integer.parseInt(data.get("year")), Store.YES));
    doc.add(new IntField("runtime", Integer.parseInt(data.get("runtime")), Store.YES));
    doc.add(new FloatField("rating", Float.parseFloat(data.get("rating")), Store.YES));

    // we do not store these fields and can't print them in the results
    doc.add(new TextField("actors", data.get("actors"), Store.NO));
    doc.add(new TextField("genre", data.get("genre"), Store.NO));

    // lastly, we use a custom field to show the term vectors in summary
    FieldType ftSummary = new FieldType();
    ftSummary.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS);
    ftSummary.setStoreTermVectors(true);
    ftSummary.setStoreTermVectorPositions(true);
    ftSummary.setTokenized(true);
    ftSummary.setStored(true);
    doc.add(new Field("summary", data.get("summary"), ftSummary));

    return doc;
}

### Load the index in batches to observe how segment creation works
Change the `batchSize` argument in the call to `loadImdbData` and observe the index folder at `./lucene/index`. Every time we create a new `IndexWriter`, Lucene creates a new segment and all documents added with the same index writer are kept in the same segments. 

Note: we deliberately turn off the merge policy to demonstrate sgements. Do not do in production!

In [20]:
void loadBatch(List<Map<String, String>> docs) throws IOException {
    IndexWriter writer = null;

    try {
        writer = getIndexWriter(false);
        for (Map<String, String> doc : docs)
            writer.addDocument(createDocument(doc));
    } finally {
        if (writer!=null) writer.close();
    }
}

void loadImdbData(int batchSize) throws IOException {
    List<Map<String, String>> collection = readCollection(fileImdbDataset);

    deleteIndex();
    // load collection in batches to show how segments work
    for (int i = 0; i < collection.size(); i += batchSize)
        loadBatch(collection.subList(i, Math.min(i + batchSize, collection.size())));
}

loadImdbData(100);

Read 1000 documents from datasets/imdb_top_1000.csv


### Merging segments
We can force Lucene to merge segments (normally, this is done less costly by the MergePolicy). Above we created 10 segments and you can observe 10 groups (compounded file format) of files in the index folder. Next, we turn on merge policy and force Lucene to produce a compact index with one segment only.

In [21]:
var writer = getIndexWriter(true);
writer.forceMerge(1);
writer.close();

### Inspecting the index

##### Documents in the index

In [22]:
void printDocuments(int num) throws IOException {
    IndexReader reader = DirectoryReader.open(getDirectory());
    StoredFields storedFields = reader.storedFields();
    for(int i = 0; i < num; i++) {
        Document doc = storedFields.document(i);
        System.out.println("Doc " + i + ": " + doc.toString());
    }
}

printDocuments(10);

Doc 0: Document<stored,indexed,tokenized<title:The Shawshank Redemption> stored,indexed,tokenized<year:1994> stored<runtime:142> stored<rating:9.3> stored,indexed,tokenized,termVector<summary:Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.>>
Doc 1: Document<stored,indexed,tokenized<title:The Godfather> stored,indexed,tokenized<year:1972> stored<runtime:175> stored<rating:9.2> stored,indexed,tokenized,termVector<summary:An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.>>
Doc 2: Document<stored,indexed,tokenized<title:The Dark Knight> stored,indexed,tokenized<year:2008> stored<runtime:152> stored<rating:9.0> stored,indexed,tokenized,termVector<summary:When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.>>
Doc 3: Document<st

##### Terms used in a document
This next code is only possible if we store the term vectors (aka document vectors) in the index. We have created a custom field for summary to demonstrate this.

In [23]:
void printDocumentVector(int docNo, String field) throws IOException {
    IndexReader reader = DirectoryReader.open(getDirectory());
    TermVectors vectors = reader.termVectors();
    Terms terms = vectors.get(docNo, field);
        
    // iterate through terms
    System.out.println("index of doc " + docNo + ": \nterm             tf   postings");
    TermsEnum termsEnum = terms.iterator();
    PostingsEnum positions = null;
    BytesRef term = null;
    while((term = termsEnum.next()) != null) {
        String termstr = term.utf8ToString(); // Get the text string of the term.
        long freq = termsEnum.totalTermFreq(); // Get the frequency of the term in the document.
        
        System.out.printf("%-12s %6d  ", termstr, freq );
        positions = termsEnum.postings(positions, PostingsEnum.POSITIONS );
        positions.nextDoc(); // you still need to move the cursor
        for(int i = 0; i < freq; i++ )
            System.out.print(" " + positions.nextPosition());
        System.out.println();
    }
    reader.close();
}

// print document (to see what happened to it)
var docNo = 999;
System.out.println("\nsummary of doc " + docNo + ": " + collection.get(docNo).get("summary") + "\n");
printDocumentVector(docNo, "summary");


summary of doc 999: A man in London tries to help a counter-espionage Agent. But when the Agent is killed, and the man stands accused, he must go on the run to save himself and stop a spy ring which is trying to steal top secret information.

index of doc 999: 
term             tf   postings
accus             1   21
agent             2   10 14
counter           1   8
espionag          1   9
go                1   24
he                1   22
help              1   6
himself           1   30
inform            1   43
kill              1   16
london            1   3
man               2   1 19
must              1   23
ring              1   35
run               1   27
save              1   29
secret            1   42
spy               1   34
stand             1   20
steal             1   40
stop              1   32
top               1   41
tri               1   4
try               1   38
when              1   12
which             1   36


##### How does the Vocabulary look like?

In [24]:
void printVocabulary(String field) throws IOException {
    IndexReader reader = DirectoryReader.open(getDirectory());
    Terms vocabulary = MultiTerms.getTerms(reader, field);
    TermsEnum termsEnum = vocabulary.iterator();
    BytesRef term = null;
    Object output[][] = new Object[(int)vocabulary.size()][];
    int i = 0;

    // get all terms from vocbulary and produce the result
    while ((term = termsEnum.next()) != null) {
        // create an array list with the terms and their frequencies
        Object[] list = new Object[]{term.utf8ToString(),termsEnum.docFreq(), termsEnum.totalTermFreq()};
        output[i++] = list;
    }

    // print summary
    System.out.println(String.format("Total documents: %d", reader.numDocs()));
    System.out.println(String.format("    Total terms: %d", vocabulary.size()));

    System.out.println(String.format("  Total sum(tf): %d", reader.getSumTotalTermFreq(field)));
    System.out.println(String.format("     Average tf: %1.2f", (float)reader.getSumTotalTermFreq(field)/reader.getSumDocFreq(field)));
    System.out.println(String.format("Average doc len: %1.2f", (float)reader.getSumTotalTermFreq(field)/reader.numDocs()));
    
    System.out.println(String.format("     Average df: %1.2f", (float)reader.getSumTotalTermFreq(field)/vocabulary.size()));
    System.out.println("\nterm             df  sum(tf)");

    // show the objects with higest df
    Arrays.sort(output, (o1, o2) -> ((Integer)o2[1]).compareTo((Integer)o1[1]));
    for (i=0; i<20; i++)
        System.out.println(String.format("%-12s %6d %8d", output[i][0], output[i][1], output[i][2]));
    reader.close();
}

printVocabulary("summary");

Total documents: 1000
    Total terms: 4463
  Total sum(tf): 17382
     Average tf: 1.03
Average doc len: 17.38
     Average df: 3.89

term             df  sum(tf)
hi              375      516
who             158      165
from            139      148
he              138      155
young           127      132
her             124      164
man             116      120
after           112      112
find            103      106
life            103      111
when            103      107
two              98      104
world            82       85
becom            73       76
year             73       78
new              70       75
famili           68       71
stori            68       68
war              68       71
up               67       67


In [25]:
printVocabulary("title");

Total documents: 1000
    Total terms: 1605
  Total sum(tf): 2251
     Average tf: 1.01
Average doc len: 2.25
     Average df: 1.40

term             df  sum(tf)
la               23       24
man              20       20
de               18       18
le               17       18
2                14       14
dai              14       14
stori            10       10
star              9        9
harri             8        8
night             8        8
war               8        8
babi              7        7
dark              7        7
dead              7        7
king              7        7
last              6        6
men               6        6
onc               6        6
potter            6        6
time              6        6


## Searching with Lucene

### We need an index searcher and query parser with the same settings as for index writers

In [26]:
IndexSearcher getIndexSearcher() throws IOException {
    return new IndexSearcher(DirectoryReader.open(getDirectory()));
}

QueryParser getQueryParser() throws IOException {
    return new MultiFieldQueryParser(new String[]{"title", "content", "genre", "actors"}, getAnalyzer());
}

QueryParser getQueryParser(String field) throws IOException {
    return new QueryParser(field, getAnalyzer());
}

### This function takes a query parser and runs a set of queries

In [27]:
void printResults(String query, TopDocs results) throws IOException {
    int rank = 1;
    System.out.println("Query: " + query);
    System.out.printf("%3s %5s %6s %6s %7s %6s   %s\n", "#", "id", "Score", "Year", "Runtime", "Rating", "Title" );
    for(ScoreDoc doc: results.scoreDocs) {
        Document document = getIndexSearcher().doc(doc.doc);
        System.out.printf("%3d %5d %6.2f %6s %7s %6s   %s\n", rank++, doc.doc, doc.score, 
            document.get("year"), document.get("runtime"), document.get("rating"), document.get("title") );
    } 
    System.out.println();
}

void searchExamples(QueryParser parser, String[] queries) throws IOException, ParseException {
    IndexSearcher searcher = getIndexSearcher();

    for(String query: queries) {
        printResults(query, searcher.search(parser.parse(query), 10));
        System.out.println();
    }
}

void searchQuery(Query query) throws IOException {
    IndexSearcher searcher = getIndexSearcher();
    TopDocs results = searcher.search(query, 10);
    printResults(query.toString(), results);
}

##### First we perform some generic queries with keywords only

In [28]:
searchExamples(getQueryParser(), new String[]{
    "star wars", 
    "drama morgan freeman", 
    "comedy"
});

Query: star wars
  #    id  Score   Year Runtime Rating   Title
  1    29   4.49   1977     121    8.6   Star Wars
  2   109   2.55   1983     131    8.3   Star Wars: Episode VI - Return of the Jedi
  3   477   2.55   2015     138    7.9   Star Wars: Episode VII - The Force Awakens
  4    16   2.30   1980     124    8.7   Star Wars: Episode V - The Empire Strikes Back
  5   931   2.27   2005     122    7.6   Lord of War
  6   493   2.22   2009     127    7.9   Star Trek
  7   903   2.22   2018     136    7.6   A Star Is Born
  8    54   1.91   2017     125    8.4   Ayla: The Daughter of War
  9    60   1.91   2018     149    8.4   Avengers: Infinity War
 10   731   1.86   2014     126    7.7   The Fault in Our Stars


Query: drama morgan freeman
  #    id  Score   Year Runtime Rating   Title
  1     0   4.46   1994     142    9.3   The Shawshank Redemption
  2   167   4.43   1992     130    8.2   Unforgiven
  3   234   4.43   2004     132    8.1   Million Dollar Baby
  4   673   4.41  

In [29]:
searchExamples(getQueryParser(), new String[]{
    "genre:drama actors:morgan actors:freeman", 
    "genre:comedy",
});

Query: genre:drama actors:morgan actors:freeman
  #    id  Score   Year Runtime Rating   Title
  1     0   4.46   1994     142    9.3   The Shawshank Redemption
  2   167   4.43   1992     130    8.2   Unforgiven
  3   234   4.43   2004     132    8.1   Million Dollar Baby
  4   673   4.41   1989     122    7.8   Glory
  5   768   4.41   2006     110    7.7   Lucky Number Slevin
  6   922   4.41   2007     114    7.6   Gone Baby Gone
  7    27   4.20   1995     127    8.6   Se7en
  8   311   2.34   1940      99    8.1   The Shop Around the Corner
  9   609   2.06   2013     161    7.8   The Hobbit: The Desolation of Smaug
 10   618   2.06   2012     169    7.8   The Hobbit: An Unexpected Journey


Query: genre:comedy
  #    id  Score   Year Runtime Rating   Title
  1    78   0.89   1964      95    8.4   Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb
  2   277   0.89   1979      94    8.1   Life of Brian
  3   417   0.89   1974     106    8.0   Young Frankenstein
 

In [30]:
searchExamples(getQueryParser(), new String[]{
    "leon",
    "l?on",
    "leon~0.6",
});

Query: leon
  #    id  Score   Year Runtime Rating   Title
  1   814   3.01   1995      99    7.7   Do lok tin si


Query: l?on
  #    id  Score   Year Runtime Rating   Title
  1    42   1.00   1994     110    8.5   L�on
  2    43   1.00   1994      88    8.5   The Lion King
  3   328   1.00   2016     118    8.0   Lion
  4   545   1.00   1968     134    7.9   The Lion in Winter
  5   814   1.00   1995      99    7.7   Do lok tin si


Query: leon~0.6
  #    id  Score   Year Runtime Rating   Title
  1    42   2.50   1994     110    8.5   L�on
  2   328   2.50   2016     118    8.0   Lion
  3   814   2.41   1995      99    7.7   Do lok tin si
  4    43   2.02   1994      88    8.5   The Lion King
  5   545   2.02   1968     134    7.9   The Lion in Winter
  6   610   1.80   2009     144    7.8   Das wei�e Band - Eine deutsche Kindergeschichte
  7   222   1.60   2013     153    8.1   Prisoners
  8   574   1.60   2016      96    7.8   Perfetti sconosciuti
  9   614   1.60   2010     116   

In [31]:
searchExamples(getQueryParser(), new String[]{
    "title:{a TO b}",
    "year:{1990 TO 2000}",
    "year:1990",
    "runtime:142"
});

Query: title:{a TO b}
  #    id  Score   Year Runtime Rating   Title
  1     4   1.00   1957      96    9.0   12 Angry Men
  2    40   1.00   1998     119    8.5   American History X
  3    54   1.00   2017     125    8.4   Ayla: The Daughter of War
  4    59   1.00   2019     181    8.4   Avengers: Endgame
  5    60   1.00   2018     149    8.4   Avengers: Infinity War
  6    71   1.00   1984     229    8.4   Once Upon a Time in America
  7    72   1.00   1981     115    8.4   Raiders of the Lost Ark
  8    74   1.00   1979     147    8.4   Apocalypse Now
  9    75   1.00   1979     117    8.4   Alien
 10    76   1.00   1971     122    8.4   Anand


Query: year:{1990 TO 2000}
  #    id  Score   Year Runtime Rating   Title
  1     0   1.00   1994     142    9.3   The Shawshank Redemption
  2     6   1.00   1994     154    8.9   Pulp Fiction
  3     7   1.00   1993     195    8.9   Schindler's List
  4     9   1.00   1999     139    8.8   Fight Club
  5    11   1.00   1994     142    8.

In [43]:
var query = new BooleanQuery.Builder()
        // .add(IntField.newExactQuery("runtime", 142), Occur.MUST)
        // .add(IntField.newExactQuery("runtime", 142), Occur.SHOULD)
        .add(IntField.newExactQuery("runtime", 142), Occur.FILTER)
        .add(new TermQuery(new Term("actors", "morgan")), Occur.SHOULD)
        .build();

searchQuery(query);

Query: #runtime:[142 TO 142] actors:morgan
  #    id  Score   Year Runtime Rating   Title
  1     0   2.20   1994     142    9.3   The Shawshank Redemption
  2    11   0.00   1994     142    8.8   Forrest Gump
  3   105   0.00   1985     142    8.3   Idi i smotri
  4   169   0.00   1988     142    8.2   Dom za vesanje
  5   506   0.00   2004     142    7.9   Harry Potter and the Prisoner of Azkaban
  6   890   0.00   2015     142    7.6   Bridge of Spies



In [39]:
var query = new BooleanQuery.Builder()
        .add(IntField.newRangeQuery("runtime", 120, 180), Occur.FILTER)
        .add(TermRangeQuery.newStringRange("year", "1990", "2000", true, false), Occur.FILTER)
        .add(new TermQuery(new Term("actors", "morgan")), Occur.SHOULD)
        .build();

searchQuery(query);

Query: #runtime:[120 TO 180] #year:[1990 TO 2000} actors:morgan
  #    id  Score   Year Runtime Rating   Title
  1     0   2.20   1994     142    9.3   The Shawshank Redemption
  2   167   2.20   1992     130    8.2   Unforgiven
  3    27   2.10   1995     127    8.6   Se7en
  4     6   0.00   1994     154    8.9   Pulp Fiction
  5     9   0.00   1999     139    8.8   Fight Club
  6    11   0.00   1994     142    8.8   Forrest Gump
  7    14   0.00   1999     136    8.7   The Matrix
  8    15   0.00   1990     146    8.7   Goodfellas
  9    24   0.00   1998     169    8.6   Saving Private Ryan
 10    44   0.00   1991     137    8.5   Terminator 2: Judgment Day



In [46]:
var query = new BooleanQuery.Builder()
        .add(new TermQuery(new Term("title", "star")), Occur.MUST)
        .add(new TermQuery(new Term("genre", "action")), Occur.SHOULD)
        .build();

searchQuery(query);

Query: +title:star genre:action
  #    id  Score   Year Runtime Rating   Title
  1    29   2.93   1977     121    8.6   Star Wars
  2   493   2.84   2009     127    7.9   Star Trek
  3   746   2.49   2013     132    7.7   Star Trek Into Darkness
  4   903   2.22   2018     136    7.6   A Star Is Born
  5   839   2.04   1982     113    7.7   Star Trek II: The Wrath of Khan
  6   109   1.97   1983     131    8.3   Star Wars: Episode VI - Return of the Jedi
  7   477   1.88   2015     138    7.9   Star Wars: Episode VII - The Force Awakens
  8   731   1.86   2014     126    7.7   The Fault in Our Stars
  9    16   1.85   1980     124    8.7   Star Wars: Episode V - The Empire Strikes Back



In [67]:
var query = new BooleanQuery.Builder()
        .add(IntField.newRangeQuery("runtime", 120, 180), Occur.FILTER)
        .add(TermRangeQuery.newStringRange("year", "1990", "2000", true, false), Occur.SHOULD)
        .add(new TermQuery(new Term("title", "shawshank")), Occur.SHOULD)
        .add(new BoostQuery(new TermQuery(new Term("actors", "morgan")), 1.5f), Occur.SHOULD)
        .build();
        
IndexSearcher searcher = getIndexSearcher();
TopDocs results = searcher.search(query, 10);
printResults(query.toString(), results);
System.out.println(searcher.explain(query, results.scoreDocs[0].doc));

Query: #runtime:[120 TO 180] year:[1990 TO 2000} title:shawshank (actors:morgan)^1.5
  #    id  Score   Year Runtime Rating   Title
  1     0   7.40   1994     142    9.3   The Shawshank Redemption
  2   167   4.31   1992     130    8.2   Unforgiven
  3    27   4.15   1995     127    8.6   Se7en
  4   234   3.31   2004     132    8.1   Million Dollar Baby
  5   673   3.31   1989     122    7.8   Glory
  6     6   1.00   1994     154    8.9   Pulp Fiction
  7     9   1.00   1999     139    8.8   Fight Club
  8    11   1.00   1994     142    8.8   Forrest Gump
  9    14   1.00   1999     136    8.7   The Matrix
 10    15   1.00   1990     146    8.7   Goodfellas

7.4037647 = sum of:
  0.0 = match on required clause, product of:
    0.0 = # clause
    1.0 = runtime:[120 TO 180]
  1.0 = year:[1990 TO 2000}
  3.0980327 = weight(title:shawshank in 0) [BM25Similarity], result of:
    3.0980327 = score(freq=1.0), computed as boost * idf * tf from:
      6.5022902 = idf, computed as log(1 + (N 

## What's next?